In [1]:
import os
import re
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from __future__ import print_function
import os
import glob
import h5py
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from torch.utils.data import Dataset,DataLoader
import torch.nn.init as init

import sklearn.metrics as metrics



## Get data from description file

In [3]:
#Print grid in readable format
def pretty_print(grid):
    for row in grid:
        print(row)

#from path of description file get the coupling grid, model data and 
#remove models with error
def process_file(path):
    with open(path,'r') as f:
        #don't consider first two lines
        f.readline()           #segmenti x x x
        f.readline()           #rotazioni x x x

        line=f.readline()      #numero pezzi x  

        #get number of fragments
        number=int(re.findall("\d+",line)[0])

        #coupling matrix
        grid=[]
        for i in range(number):
            line=f.readline()
            ret=re.findall('-1|0|1',line)
            grid.append(list(map(int,ret)))

        #model data
        model_names=[]
        non_valid_indeces=[]
        for m in range(number):
            f.readline()        #blank line
            name=f.readline()   #model name
            mesh=f.readline()   #mesh n
            f.readline()        #external n
            f.readline()        #internal n

            #if mesh=0 I don't consider the element
            mesh_n=int(mesh.rstrip().split(" ")[1]) 
            if mesh_n!=0: 
                #try create file name
                file_name= name.rstrip().replace(".","_")
                model_names.append(file_name)          
            else:
                #saving indices to remove later
                non_valid_indeces.append(m)

        #removing elements from grid
        #sorted in reverse to avoid wrong index
        for index in sorted(non_valid_indeces, reverse=True):
            #remove row
            del grid[index]
            #remove columns
            for row in grid:
                del row[index]
        
        return grid,model_names


In [4]:
#get a set of fragments (i.e. a subfolder)
def get_set(folder,verbose=True):
    folder_path=os.path.join(main_path,folder)
    models_file=[]

    #files are either text files or models
    for file in os.listdir(folder_path):
        if '.txt' in file:
            description_file=file
        elif '.stl' in file:
            #not used, they are not in the same order of the file
            models_file.append(file)

    #parsing description file
    full_description_file=os.path.join(folder_path,description_file)
    grid,model_names=process_file(full_description_file)

    #get path of models of current set
    model_prefix=folder.replace("generatedTest_","")
    complete_models_path=[]
    for i in range(len(model_names)):
        #e.g. 2021_11_29_10_00_40_Cube_001.stl
        correct_model_name=f"{model_prefix}_{model_names[i]}.stl"
        #saving only names, could be useful but now not used
        model_names[i]=correct_model_name

        complete_models_path.append(os.path.join(folder_path,correct_model_name))

    #create set and put into list of sets
    set={"models":complete_models_path,"grid":grid}
    
    if verbose:
        print("A set: \n")
        print("Models: ",set["models"])
        print("Grid: ")
        pretty_print(set["grid"])

    return set

In [5]:
#here i save all the sets of fragments
sets=[]

#root folder
main_path="produzione_29112021"

In [6]:
#example set
for folder in os.listdir(main_path):
    #check only folders, not files
    if '.' not in folder:
        set1=get_set(folder)
        sets.append(set1)
        break 
          

A set: 

Models:  ['produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_001.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_002.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_003.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_004.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_005.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_006.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_007.stl']
Grid: 
[-1, 1, 0, 0, 1, 1, 1, 1]
[1, -1, 1, 0, 1, 1, 1, 0]
[0, 1, -1, 1, 1, 1, 1, 0]
[0, 0, 1, -1, 1, 0, 1, 0]
[1, 1, 1, 1, -1, 0, 1, 1]
[1, 1, 1, 0, 0, -1, 1, 0]
[1, 1, 1, 1, 1, 1, -1, 1]
[1, 0, 0, 0, 1, 0, 1, -1]


## Visualize 3D models and pointCloud

In [7]:
#visualize some models and point clouds
for set in sets:
    for path in set["models"]:
        mesh = o3d.io.read_triangle_mesh(path)
        pointcloud = mesh.sample_points_poisson_disk(1000)

        # you can plot and check
        #o3d.visualization.draw_geometries([mesh])
        #o3d.visualization.draw_geometries([pointcloud])
        
        break
    break

# Data

## Our data

In [8]:
#here i save all the sets of fragments
sets=[]

#root folder
main_path="produzione_29112021"

In [9]:
#Create all the possible pairs of fragments
#duplicates are not considered, e.g.: (i,j) - (j,i)
def create_pairs(num):
    lista = []
    for i in range(num):
        for j in range(i+1, num):
            lista.append((i, j))

    return lista

print(create_pairs(5))


[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [16]:
def create_dataset(main_path,num_points,max_elements=-1):
    
    num_elements=0
    num_pairs=0

    all_data=[]
    all_labels=[]
    sets=[]
    print("Creating dataset...")
    print(f"Each point cloud is sampled with {num_points} points")
    num_totali=0
    num_adiacenti=0

    for folder in os.listdir(main_path):
        if max_elements>0 and num_elements>max_elements:
            print("maximum number of objects reached, finishing dataset")
            break
        #check only folders, not files
        if '.' not in folder:
            fragment_set=get_set(folder,verbose=False)  #set of fragments of one model
            sets.append(fragment_set)
            set_pointcloud=[]
            for path in fragment_set["models"]:
                num_elements+=1
                mesh = o3d.io.read_triangle_mesh(path)
                pointcloud = mesh.sample_points_poisson_disk(num_points)
                set_pointcloud.append(pointcloud)

            #get indeces of pairs
            pairs=create_pairs(len(fragment_set["models"]))
            
            for pair in pairs:
                num_pairs+=1
                idx1,idx2=pair

                label=fragment_set["grid"][idx1][idx2]
                pair=[ np.asarray(set_pointcloud[idx1].points) ,  np.asarray(set_pointcloud[idx2].points) ]

                num_totali+=1
                if label==1:
                    num_adiacenti+=1

                all_data.append(pair)
                all_labels.append(label)
    print("totali: ",num_totali," adiacenti: ",num_adiacenti)

    print(f"Dataset contains {num_elements} fragments--> {num_pairs} pairs ")
    
    return np.array(all_data),np.array(all_labels)

#used in class ModelNet40
#for now we don't use it
def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud         


class MyDataset(Dataset):
    def __init__(self,main_path, num_points, partition='train',max_elements=-1):
        self.data, self.label = create_dataset(main_path,num_points,max_elements)
        print("shape of element is: ",self.data.shape)
        self.num_points = num_points
        self.partition = partition        

    def __getitem__(self, item):
        pointclouds = self.data[item]
        label = self.label[item]
        
        #if self.partition == 'train':
            #pointclouds = translate_pointcloud(pointcloud)
            #np.random.shuffle(pointcloud)

        return pointclouds, label

    def __len__(self):
        return self.data.shape[0]


In [17]:
#example dataloader

train_loader = DataLoader(MyDataset(partition='train',num_points=50,main_path=main_path,max_elements=2), num_workers=0,
                              batch_size=5, shuffle=True, drop_last=True)
print(len(train_loader))
for data, label in train_loader:
    print(data.shape)
    print(label.shape)

    

Creating dataset...
Each point cloud is sampled with 50 points
maximum number of objects reached, finishing dataset
totali:  28  adiacenti:  19
Dataset contains 8 fragments--> 28 pairs 
shape of element is:  (28, 2, 50, 3)
5
torch.Size([5, 2, 50, 3])
torch.Size([5])
torch.Size([5, 2, 50, 3])
torch.Size([5])
torch.Size([5, 2, 50, 3])
torch.Size([5])
torch.Size([5, 2, 50, 3])
torch.Size([5])
torch.Size([5, 2, 50, 3])
torch.Size([5])


# Util

In [12]:

def cal_loss(pred, gold, smoothing=False):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    #we do not use label smoothing

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.binary_cross_entropy(pred, gold, reduction='mean')

    return loss

#class to print on file
class IOStream():
    def __init__(self, path):
        self.f = open(path, 'a')

    def cprint(self, text):
        print(text)
        self.f.write(text+'\n')
        self.f.flush()

    def close(self):
        self.f.close()


# Model.py

In [13]:

def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx

#aggiunto parametro device in modo da usare cpu se non gpu
def get_graph_feature(x, k=20,device="cpu", idx=None, dim9=False):
    batch_size = x.size(0)
    num_points = x.size(2)
    #print(x.shape)
    x = x.view(batch_size, -1, num_points)
    #print(x.shape)

    if idx is None:
        if dim9 == False:
            idx = knn(x, k=k)   # (batch_size, num_points, k)
        else:
            idx = knn(x[:, 6:], k=k)
    device = torch.device(device)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
  
    return feature      # (batch_size, 2*num_dims, num_points, k)


In [14]:

class PointNet(nn.Module):
    def __init__(self, args, output_channels=40):
        super(PointNet, self).__init__()
        self.args = args
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv5 = nn.Conv1d(128, args["emb_dims"], kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])
        self.linear1 = nn.Linear(args["emb_dims"], 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout()
        self.linear2 = nn.Linear(512, output_channels)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.adaptive_max_pool1d(x, 1).squeeze()
        x = F.relu(self.bn6(self.linear1(x)))
        x = self.dp1(x)
        x = self.linear2(x)
        return x

In [15]:

class DGCNN_cls(nn.Module):
    def __init__(self, args, output_channels=1):
        super(DGCNN_cls, self).__init__()
        self.args = args
        self.device="cuda" if self.args["cuda"] else "cpu"
        self.k = args["k"]
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args["emb_dims"], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(args["emb_dims"]*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=args["dropout"])
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=args["dropout"])
        self.linear3 = nn.Linear(256, 32)

        #final classification layer
        self.linear_classification = nn.Linear(64, output_channels)


    
    def step(self,x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k,device=self.device)      # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
        x = self.conv1(x)                       # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k,device=self.device)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv2(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k,device=self.device)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = get_graph_feature(x3, k=self.k,device=self.device)     # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
        x = self.conv4(x)                       # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
        x4 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

        x = torch.cat((x1, x2, x3, x4), dim=1)  # (batch_size, 64+64+128+256, num_points)

        x = self.conv5(x)                       # (batch_size, 64+64+128+256, num_points) -> (batch_size, emb_dims, num_points)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x = torch.cat((x1, x2), 1)              # (batch_size, emb_dims*2)

        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2) # (batch_size, emb_dims*2) -> (batch_size, 512)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2) # (batch_size, 512) -> (batch_size, 256)
        x = self.dp2(x)
        x = self.linear3(x)                                             # (batch_size, 256) -> (batch_size, output_channels)
        return x

    def forward(self, x):
        #first pointcloud
        first=x[:,0,:,:]
        second=x[:,1,:,:]

        #compute DGCNN output from both pointclouds 
        ret1=self.step(first)      
        ret2=self.step(second)

        #combine output and use classifier
        ret=torch.cat((ret1,ret2),dim=1)
        x=self.linear_classification(ret)

        return x


# Main

In [16]:
def _init_():
    if not os.path.exists('outputs'):
        os.makedirs('outputs')
    if not os.path.exists('outputs/'+args["exp_name"]):
        os.makedirs('outputs/'+args["exp_name"])
    if not os.path.exists('outputs/'+args["exp_name"]+'/'+'models'):
        os.makedirs('outputs/'+args["exp_name"]+'/'+'models')
    os.system('cp main_cls.py outputs'+'/'+args["exp_name"]+'/'+'main_cls.py.backup')
    os.system('cp model.py outputs' + '/' + args["exp_name"] + '/' + 'model.py.backup')
    os.system('cp util.py outputs' + '/' + args["exp_name"] + '/' + 'util.py.backup')
    os.system('cp data.py outputs' + '/' + args["exp_name"] + '/' + 'data.py.backup')


In [ ]:

def train(args, io):
    # num_workers=0 to make it work on windows
    train_loader = DataLoader(MyDataset(main_path, num_points=args["num_points"], partition='train',max_elements=args["max_elements"]), num_workers=0,
                              batch_size=args["batch_size"], shuffle=True, drop_last=True)
    test_loader = DataLoader(MyDataset(main_path, num_points=args["num_points"], partition='test',max_elements=args["max_elements"]), num_workers=0,
                             batch_size=args["test_batch_size"], shuffle=True, drop_last=False)

    device = torch.device("cuda" if args["cuda"] else "cpu")

    #Try to load models
    if args["model"] == 'pointnet':
        model = PointNet(args).to(device)
    elif args["model"] == 'dgcnn':
        model = DGCNN_cls(args).to(device)
    else:
        raise Exception("Not implemented")

    print("Model loaded!")
    #print(str(model))

    model = nn.DataParallel(model)
    print("Let's use", torch.cuda.device_count(), "GPUs!")

    if args["use_sgd"]:
        print("Use SGD")
        opt = optim.SGD(model.parameters(), lr=args["lr"]*100,
                momentum=args["momentum"], weight_decay=1e-4)
    else:
        print("Use Adam")
        opt = optim.Adam(model.parameters(), lr=args["lr"], weight_decay=1e-4)

    if args["scheduler"] == 'cos':
        scheduler = CosineAnnealingLR(opt, args["epochs"], eta_min=1e-3)
    elif args["scheduler"] == 'step':
        scheduler = StepLR(opt, step_size=20, gamma=0.7)
    
    criterion = cal_loss

    best_test_acc = 0
    print(args)
    for epoch in range(args["epochs"]):
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        print("Starting epoch ",epoch)
        for data, label in train_loader:
            #print("get data")
            data, label = data.to(device), label.to(device).squeeze()
            #print(label)
            data=data.float()
            label=label.float()
            data = data.permute(0,1,3,2)
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            logits=torch.squeeze(torch.sigmoid(logits))
            loss = criterion(logits, label)
            
            loss.backward()
            opt.step()
            
            #preds = logits.max(dim=1)[1] #not needed, we have only one value for each sample
            preds=logits.int()
            count += batch_size
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
        
        if args["scheduler"] == 'cos':
            scheduler.step()
        elif args["scheduler"] == 'step':
            if opt.param_groups[0]['lr'] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]['lr'] < 1e-5:
                for param_group in opt.param_groups:
                    param_group['lr'] = 1e-5

        
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,
                                                                                 train_loss*1.0/count,
                                                                                 metrics.accuracy_score(
                                                                                     train_true, train_pred),
                                                                                 metrics.balanced_accuracy_score(
                                                                                     train_true, train_pred))
        io.cprint(outstr)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            data=data.float()
            label=label.float()

            data = data.permute(0,1,3,2)
            batch_size = data.size()[0]
            logits = model(data)
            logits=torch.squeeze(torch.sigmoid(logits))
            loss = criterion(logits, label)
            
            #preds = logits.max(dim=1)[1] #not needed, we have only one value for each sample
            preds=logits.int()

            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        test_acc = metrics.accuracy_score(test_true, test_pred)
        avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,
                                                                              test_loss*1.0/count,
                                                                              test_acc,
                                                                              avg_per_class_acc)
        io.cprint(outstr)

        ''' 
        #save model does not work, need file created, to do
        if test_acc >= best_test_acc:
            best_test_acc = test_acc
            torch.save(model.state_dict(), 'outputs/%s/models/model.t7' % args["use_sgd"])'''


In [18]:

def test(args, io):
    test_loader = DataLoader(MyDataset(partition='test', num_points=args["num_points"]),
                             batch_size=args["test_batch_size"], shuffle=True, drop_last=False)

    device = torch.device("cuda" if args["cuda"] else "cpu")

    #Try to load models
    if args["mdoel"] == 'pointnet':
        model = PointNet(args).to(device)
    elif args["mdoel"] == 'dgcnn':
        model = DGCNN_cls(args).to(device)
    else:
        raise Exception("Not implemented")

    model = nn.DataParallel(model)
    
    model.load_state_dict(torch.load(args["model_path"]))
    model = model.eval()
    test_acc = 0.0
    count = 0.0
    test_true = []
    test_pred = []
    for data, label in test_loader:

        data, label = data.to(device), label.to(device).squeeze()
        data = data.permute(0, 2, 1)
        batch_size = data.size()[0]
        logits = model(data)
        #preds = logits.max(dim=1)[1]
        preds=logits.int()
        test_true.append(label.cpu().numpy())
        test_pred.append(preds.detach().cpu().numpy())
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    test_acc = metrics.accuracy_score(test_true, test_pred)
    avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
    outstr = 'Test :: test acc: %.6f, test avg acc: %.6f'%(test_acc, avg_per_class_acc)
    io.cprint(outstr)



In [19]:


if __name__ == "__main__":
    
    args={
        "exp_name": "exp",          # Name of the experiment
        "model": "dgcnn",           # Model to use, [pointnet, dgcnn]
        "dataset": "modelnet40",    # Dataset to use, we don't use this
        "batch_size": 16,           # Size of batch
        "test_batch_size": 16,      # Size of batch
        "epochs": 50,              # number of episode to train
        "use_sgd": False,            # Use SGD
        "lr": 0.001,                # learning rate (default: 0.001, 0.1 if using sgd)
        "momentum": 0.9,            # SGD momentum (default: 0.9)
        "scheduler": "cos",         # Scheduler to use, [cos, step]
        "no_cuda": False,           # enables CUDA training
        "seed": 1,                  # random seed (default: 1)
        "eval": False,              # evaluate the model
        "num_points": 100,         # num of points to use
        "dropout": 0.5,             # initial dropout rate
        "emb_dims": 1024,           # Dimension of embeddings
        "k": 20,                    # Num of nearest neighbors to use
        "model_path": "",           # Pretrained model path
        "max_elements": 30,         # Num of 3D fragments to consider
    }

    _init_()

    io = IOStream('outputs/' + args["exp_name"] + '/run.log')
    io.cprint(str(args))
    
    #set up GPU
    args["cuda"] = not args["no_cuda"] and torch.cuda.is_available()
    torch.manual_seed(args["seed"])
    if args["cuda"]:
        io.cprint(
            'Using GPU : ' + str(torch.cuda.current_device()) + ' from ' + str(torch.cuda.device_count()) + ' devices')
        torch.cuda.manual_seed(args["seed"])
    else:
        io.cprint('Using CPU')

    if not args["eval"]:
        train(args, io)
    else:
        test(args, io)


{'exp_name': 'exp', 'model': 'dgcnn', 'dataset': 'modelnet40', 'batch_size': 16, 'test_batch_size': 16, 'epochs': 50, 'use_sgd': False, 'lr': 0.001, 'momentum': 0.9, 'scheduler': 'cos', 'no_cuda': False, 'seed': 1, 'eval': False, 'num_points': 100, 'dropout': 0.5, 'emb_dims': 1024, 'k': 20, 'model_path': '', 'max_elements': 30}
Using CPU
Creating dataset...
Each point cloud is sampled with 100 points
maximum number of objects reached, finishing dataset
Dataset contains 51 fragments--> 435 pairs 
Creating dataset...
Each point cloud is sampled with 100 points
maximum number of objects reached, finishing dataset
Dataset contains 51 fragments--> 435 pairs 
Model loaded!
Let's use 0 GPUs!
Use Adam
{'exp_name': 'exp', 'model': 'dgcnn', 'dataset': 'modelnet40', 'batch_size': 16, 'test_batch_size': 16, 'epochs': 50, 'use_sgd': False, 'lr': 0.001, 'momentum': 0.9, 'scheduler': 'cos', 'no_cuda': False, 'seed': 1, 'eval': False, 'num_points': 100, 'dropout': 0.5, 'emb_dims': 1024, 'k': 20, 'mode

C:\Users\alessio\Anaconda3\envs\EAI\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train 0, loss: 0.620520, train acc: 0.648148, train avg acc: 0.500000
Test 0, loss: 0.589304, test acc: 0.648276, test avg acc: 0.500000
Starting epoch  1
Train 1, loss: 0.587883, train acc: 0.648148, train avg acc: 0.500000


KeyboardInterrupt: 